In [81]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from tabulate import tabulate

In [82]:
with open('new_train.csv', 'r') as csvfile:
    df = pd.read_csv(csvfile, dtype={'booking_bool':bool,
                                     'prop_country_id':np.int32, 
                                     'prop_starrating':np.int32, 'prop_review_score':np.float,
                                     'prop_location_score1':np.float,
                                     'prop_location_score2':np.float, 'price_usd':np.float,
                                     'price_usd':np.float, 'total_price':np.float, 'price_per_pers':np.float,
                                     'srch_query_affinity_score':np.float, 'orig_destination_distance':np.float,
                                     'position':np.int32, 'click_bool':bool})


In [83]:
target = ['booking_bool', 'click_bool']

### My selection of features after applying the Feature combination with worst case scenario algorithm
(Mind that some of these features are not selected in the final step when creating new_train.csv in the notebook that is currently on git)

In [84]:
data =  ['srch_id','prop_country_id', 'prop_location_score1','prop_location_score2','diff_hist_price',
                'usd_diff', 'star_diff', 'srch_query_affinity_score', 
                'prop_review_score','position','price_rank','stars_rank','score_rank',
        'random_bool', 'price_usd', 'total_price','prop_id_counts', 'comp2_rate','comp5_rate','comp8_rate']

### Take a sample by using 10% of the srch_id


In [85]:
sample_srch_id = np.random.choice(df.srch_id.unique(), size= int(0.1 * len(df.srch_id.unique())))
df_sample = df[df.srch_id.isin(sample_srch_id)]
print len(df_sample)
print df_sample['click_bool'].corr(df['click_bool'])
print df_sample['booking_bool'].corr(df['booking_bool'])

449457
1.0
1.0


In [86]:
df = df.drop(df_sample.index)

In [87]:
X = df_sample[data]
y = df_sample[target]
pred = df[data]

### Decision Tree 
(I think the negative data should be downsampled a bit for this too)

In [88]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)
prediction = clf.predict(pred)
search = 1
print any(e[0] == search for e in prediction)
#print cross_val_score(clf, X, y, cv=10, scoring='accuracy').mean()

True


In [89]:
from collections import Counter
Counter(x for xs in prediction for x in set(xs))
head = ['booking_bool','click_bool']
prediction_df = pd.DataFrame(prediction, columns=head)

diff =  np.subtract(prediction_df.click_bool, np.int32(df.click_bool))
print "Total number of clicked predictions: ", len(prediction_df[prediction_df.click_bool == 1])
print "Number of correct clicked/non-clicked predictions: ", np.count_nonzero(diff == 0)
print "Number of false positives clicked", np.count_nonzero(diff == 1)
print "Number of false negatives clicked", np.count_nonzero(diff == -1)

diff2 =  np.subtract(prediction_df.booking_bool, np.int32(df.booking_bool))
print "Total number of booked predictions: ", len(prediction_df[prediction_df.booking_bool == 1])
print "Number of correct booked/non-booked predictions: ", np.count_nonzero(diff2 == 0)
print "Number of false positives booked", np.count_nonzero(diff2 == 1)
print "Number of false negatives booked", np.count_nonzero(diff2 == -1)


Total number of clicked predictions:  237326
Number of correct clicked/non-clicked predictions:  3875346
Number of false positives clicked 212447
Number of false negatives clicked 167544
Total number of booked predictions:  148000
Number of correct booked/non-booked predictions:  4016298
Number of false positives booked 133046
Number of false negatives booked 105993


In [90]:
dictImp  = zip(X.head(0), clf.feature_importances_)
print tabulate(dictImp)

-------------------------  ----------
srch_id                    0.10505
prop_country_id            0.0318835
prop_location_score1       0.070066
prop_location_score2       0.0934204
diff_hist_price            0.107625
usd_diff                   0.0562137
star_diff                  0.0192667
srch_query_affinity_score  0.0166066
prop_review_score          0.0271887
position                   0.0652986
price_rank                 0.0508573
stars_rank                 0.0498093
score_rank                 0.0585629
random_bool                0.00824915
price_usd                  0.0514025
total_price                0.0682337
prop_id_counts             0.0891759
comp2_rate                 0.00903522
comp5_rate                 0.0123404
comp8_rate                 0.0097146
-------------------------  ----------


### Downsample negative data for the following classifiers
(They give very bad results if the data is extremely unbalanced)

In [91]:
grouped  = df_sample[df_sample.click_bool == False].groupby('srch_id')
grouped = grouped.apply(lambda x: x.sample(frac = 0.2))
grouped = grouped.apply(lambda x: x) 
df_sample = df_sample.drop(df_sample[df_sample.click_bool == False].index).append(grouped)

In [92]:
X = df_sample[data]
y = df_sample[target]
pred = df[data]

### KNeighbors - bad

In [93]:
knn  = KNeighborsClassifier(n_neighbors  = 5)
knn.fit(X,y)
prediction = knn.predict(pred)
#print cross_val_score(knn, X, y, cv = 10, scoring = 'accuracy').mean()

In [94]:
Counter(x for xs in prediction for x in set(xs))
head = ['booking_bool','click_bool']
prediction_df = pd.DataFrame(prediction, columns=head)

diff =  np.subtract(prediction_df.click_bool, np.int32(df.click_bool))
print "Total number of clicked predictions: ", len(prediction_df[prediction_df.click_bool == 1])
print "Number of correct clicked/non-clicked predictions: ", np.count_nonzero(diff == 0)
print "Number of false positives clicked", np.count_nonzero(diff == 1)
print "Number of false negatives clicked", np.count_nonzero(diff == -1)

diff2 =  np.subtract(prediction_df.booking_bool, np.int32(df.booking_bool))
print "Total number of booked predictions: ", len(prediction_df[prediction_df.booking_bool == 1])
print "Number of correct booked/non-booked predictions: ", np.count_nonzero(diff2 == 0)
print "Number of false positives booked", np.count_nonzero(diff2 == 1)
print "Number of false negatives booked", np.count_nonzero(diff2 == -1)

Total number of clicked predictions:  146825
Number of correct clicked/non-clicked predictions:  3930843
Number of false positives clicked 139448
Number of false negatives clicked 185046
Total number of booked predictions:  24902
Number of correct booked/non-booked predictions:  4111130
Number of false positives booked 24081
Number of false negatives booked 120126


### Gradient Boosting - best performance

In [95]:
y_GBC = y.click_bool
clf = GradientBoostingClassifier(max_depth = 2)
clf.fit(X, y_GBC)
prediction = clf.predict(pred)
#print cross_val_score(clf, X,y_GBC, cv=10, scoring='accuracy').mean()

In [96]:
print "Total number of clicked predictions:", np.count_nonzero(prediction == 1)
diff =  np.subtract(prediction, np.int32(df.click_bool))
print "Total number of correct predictions:", np.count_nonzero(diff == 0)
print "Number of false positives:", np.count_nonzero(diff == 1)
print "Number of false negatives:", np.count_nonzero(diff == -1)


Total number of clicked predictions: 166855
Total number of correct predictions: 3974703
Number of false positives: 127533
Number of false negatives: 153101


In [97]:
clf = GradientBoostingClassifier(max_depth = 2)
clf.fit(X, y.booking_bool)
prediction = clf.predict(pred)
#print cross_val_score(clf, X, y.booking_bool, cv=10, scoring='accuracy').mean()

In [98]:
print "Total number of booked predictions:", np.count_nonzero(prediction == 1)
diff =  np.subtract(prediction, np.int32(df.booking_bool))
print "Total number of correct predictions:", np.count_nonzero(diff == 0)
print "Number of false positives:", np.count_nonzero(diff == 1)
print "Number of false negatives:", np.count_nonzero(diff == -1)


Total number of booked predictions: 89811
Total number of correct predictions: 4089081
Number of false positives: 67560
Number of false negatives: 98696


In [99]:
dictImp  = zip(X.head(0), clf.feature_importances_)
print tabulate(dictImp)


-------------------------  ----------
srch_id                    0
prop_country_id            0
prop_location_score1       0.0794228
prop_location_score2       0.0935354
diff_hist_price            0.0137857
usd_diff                   0.0806949
star_diff                  0.025223
srch_query_affinity_score  0.0038279
prop_review_score          0.0463426
position                   0.189496
price_rank                 0.195092
stars_rank                 0.0735313
score_rank                 0.0312322
random_bool                0.136868
price_usd                  0.0192973
total_price                0.0101212
prop_id_counts             0.00152932
comp2_rate                 0
comp5_rate                 0
comp8_rate                 0
-------------------------  ----------


### Downsample negative data even more for random forest

In [100]:
grouped  = df_sample[df_sample.click_bool == False].groupby('srch_id')
grouped = grouped.apply(lambda x: x.sample(frac = 0.7))
grouped = grouped.apply(lambda x: x) 
df_sample = df_sample.drop(df_sample[df_sample.click_bool == False].index).append(grouped)

In [101]:
X = df_sample[data]
y = df_sample[target]
pred = df[data]

### Random Forest 
(takes a LOOOOT to compute and I've tested a few parameters for the sample fraction above, but clearly haven't found the right one yet - I've also tried 0.1, 0.5 which led to too many clicks and bookings predicted. If the negative data is not downsampled, it predicts 0 bookings and very few clicks)

In [102]:
clf = RandomForestClassifier(random_state = 0, n_estimators = 1000)
clf.fit(X, y)
prediction = clf.predict(pred)
#print cross_val_score(clf, X, y, cv=10, scoring='accuracy').mean()


In [103]:
Counter(x for xs in prediction for x in set(xs))
head = ['booking_bool','click_bool']
prediction_df = pd.DataFrame(prediction, columns=head)

diff =  np.subtract(prediction_df.click_bool, np.int32(df.click_bool))
print "Total number of clicked predictions: ", len(prediction_df[prediction_df.click_bool == 1])
print "Number of correct clicked/non-clicked predictions: ", np.count_nonzero(diff == 0)
print "Number of false positives clicked", np.count_nonzero(diff == 1)
print "Number of false negatives clicked", np.count_nonzero(diff == -1)

diff2 =  np.subtract(prediction_df.booking_bool, np.int32(df.booking_bool))
print "Total number of booked predictions: ", len(prediction_df[prediction_df.booking_bool == 1])
print "Number of correct booked/non-booked predictions: ", np.count_nonzero(diff2 == 0)
print "Number of false positives booked", np.count_nonzero(diff2 == 1)
print "Number of false negatives booked", np.count_nonzero(diff2 == -1)


Total number of clicked predictions:  333702
Number of correct clicked/non-clicked predictions:  3853946
Number of false positives clicked 271335
Number of false negatives clicked 130056
Total number of booked predictions:  150907
Number of correct booked/non-booked predictions:  4046093
Number of false positives booked 119602
Number of false negatives booked 89642


In [104]:
dictImp  = zip(X.head(0), clf.feature_importances_)
print tabulate(dictImp)

-------------------------  ----------
srch_id                    0.0721547
prop_country_id            0.0284137
prop_location_score1       0.0635024
prop_location_score2       0.0817507
diff_hist_price            0.0748979
usd_diff                   0.0614441
star_diff                  0.0217553
srch_query_affinity_score  0.0102795
prop_review_score          0.0267466
position                   0.147498
price_rank                 0.0632593
stars_rank                 0.0484986
score_rank                 0.05485
random_bool                0.0292861
price_usd                  0.0599001
total_price                0.0614174
prop_id_counts             0.0692293
comp2_rate                 0.0079565
comp5_rate                 0.00961665
comp8_rate                 0.00754364
-------------------------  ----------


In [105]:
len(df_sample[df_sample.click_bool == True])

20259